In [226]:
import requests
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import spotipy.util as util
from bs4 import BeautifulSoup as BS
import spotipy_cred as cred

## Spotify API calls utilizing Spotipy

In [227]:
client_id = cred.id
client_secret = cred.secret

username = cred.user
Playlist_id = '75yDQp4uFy8iUyyMCQ8IZ3'

Redirect_uri = 'localhost:8888/callback/'

scope = 'playlist-modify-public'

In [228]:
#credentials for Client Credentials Flow

credentials = spotipy.oauth2.SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)


In [200]:
#Obtain API token for Authorization Code Flow

util.prompt_for_user_token(username = username ,scope = 'playlist-modify-public',
client_id= client_id ,client_secret= client_secret,redirect_uri='localhost:8888/callback/')



## WebScraping Pitchfork

In [4]:
url1 = 'https://www.albumoftheyear.org/genre/27-punk-rock/all/pitchfork/'
url2 = 'https://www.albumoftheyear.org/genre/27-punk-rock/all/pitchfork/2/'

In [5]:
def request_1(url):
    response = requests.get(url)
    print(response.status_code)
    soup = BS(response.content, 'html.parser')
    return soup

In [8]:
page1 = request_1(url1)
page2 = request_1(url2)

200
200


In [189]:
page1

<!DOCTYPE html>

<html lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<title>Pitchfork's Best Punk Rock Albums of All Time - Album of The Year</title>
<meta content="Pitchfork's Best Punk Rock Albums of All Time. View reviews, ratings, news &amp; more regarding your favorite band." name="Description"/>
<meta content="Album of The Year" property="og:site_name">
<meta content="Pitchfork's Best Punk Rock Albums of All Time" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://cdn.albumoftheyear.org/album/6047-pink-flag.jpg" property="og:image"/>
<link href="https://cdn.albumoftheyear.org/album/6047-pink-flag.jpg" rel="image_src"/>
<meta content="8605207" property="fb:admins"/>
<meta content="387133090556" property="fb:app_id"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@aoty" name="twitter:site"/>
<m

In [7]:
#parsing response

def artist_extract(soup_object):
    artist_album = []
    page = soup_object.find_all('div', class_ = 'albumListCover')
    for i in range(len(page)):
        for x in page[i]('img'):
            artist_album.append(x['alt'])
    return artist_album

In [10]:
artist_album1 = artist_extract(page1)
artist_album2 = artist_extract(page2)

In [12]:
artist_album2

['Titus Andronicus - The Most Lamentable Tragedy',
 "The Replacements - Don't Tell a Soul",
 'The Jam - This Is the Modern World',
 'Beach Slang - The Things We Do To Find People Who Feel Like Us',
 'The Distillers - Coral Fang',
 'Wire - Red Barked Tree',
 'Royal Headache - High',
 'Screaming Females - Ugly',
 'Iron Chic - The Constant One',
 'The Replacements - Hootenanny',
 'Screaming Females - Castle Talk',
 'Screaming Females - Chalk Tape',
 'Protomartyr - No Passion All Technique',
 'Dropkick Murphys - Do or Die',
 'Joyce Manor - Never Hungover Again',
 'Sleater-Kinney - Sleater-Kinney',
 'Jawbreaker - Unfun',
 'The So So Glos - Blowout',
 'Alice Bag - Alice Bag',
 'Wire - Send',
 'Wire - Object 47',
 'Against Me! - Transgender Dysphoria Blues',
 'Against Me! - Shape Shift With Me',
 'Alice Bag - Blueprint',
 'Single Mothers - Negative Qualities']

In [19]:
 def get_album_info(list_of_albums):
    """Takes a list of albums and makes spotify API call"""
    results = []
    for album in list_of_albums:
        results.append(spotify.search(q=f'{album}', type="album", limit=1))
    return results

In [235]:
json_1 = get_album_info(artist_album1)

In [236]:
json_2 = get_album_info(artist_album2)

In [161]:
json_2[0]

{'albums': {'href': 'https://api.spotify.com/v1/search?query=Wire+-+Pink+Flag&type=album&offset=0&limit=1',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2i8ynmFv4qgRksyDlBgi6d'},
      'href': 'https://api.spotify.com/v1/artists/2i8ynmFv4qgRksyDlBgi6d',
      'id': '2i8ynmFv4qgRksyDlBgi6d',
      'name': 'Wire',
      'type': 'artist',
      'uri': 'spotify:artist:2i8ynmFv4qgRksyDlBgi6d'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
     'LB',
     'LI',
     'LT',
     'LU',
     'LV'

In [92]:
len(json_1)

25

In [237]:
album_id_1 = get_album_ids(json_1)


In [238]:
album_id_2 = get_album_ids(json_2)

In [135]:
def get_album_ids(json_return):
    album_ids = []
    contents = []
    for items in json_return:
        contents = items['albums']
        for item in contents['items']:
            album_ids.append(item['uri'])
    return album_ids

In [130]:
for items in json_1:
    new_list = items['albums']
    for item in new_list['items']:
        print(item['uri'])
        
        
#         print(i['href'])
#         for uri in i['items']:
#             print(uri['uri'])

spotify:album:6tcFu0TqSurn0s4unYDXSO
spotify:album:31XXN8TRj92V1HB7Wq6LcL
spotify:album:1ofMLjpy6bhkxpOcD5y436
spotify:album:3YIO1RBJiQoGLimgK6XisN
spotify:album:23MOdUJkZlG971DVuEINMr
spotify:album:3Tm6OMfXk7PffAEt4NAkmX
spotify:album:0T9tIRy1SjZTdTcMFhufu9
spotify:album:63mOF4rcNutr1Aaj9QPgJK
spotify:album:4fJeL4hdOewP135z0z0xhO
spotify:album:0pBqLz20Olwl0JVODWwyoI
spotify:album:5xPSi0aOjs3cbObM9ZTOd1
spotify:album:6HlGzD3T7aL3Zr7w7QcNTn
spotify:album:52FakbyRtaf6X8UzKxJfoh
spotify:album:5EKDBIKZ37Idq0QiXG2qzk
spotify:album:6cPoZUF1MLvQvXkfDGHwsf
spotify:album:6ArjrXMlLegKiAPOp34K58
spotify:album:4TAEAcuqfrU87gVTO6LoCP
spotify:album:6mtyAknyjUbmwCP0Pjlk3k
spotify:album:3MaBkywrJ1juBymiak7R2k
spotify:album:4mBHxNHbABZPnckTpL2OSN
spotify:album:1hxLe975qGYhzosF74wLxC
spotify:album:4MuH5XuLZFuDc6kB2CSapu
spotify:album:3lqqdqQPSFFujnoYAQbcKy
spotify:album:166nVB36rcSGaol8O8AXfW
spotify:album:1J8MyASMXsb0Y1bPeCwTKg


In [137]:
album_id_1

['spotify:album:6tcFu0TqSurn0s4unYDXSO',
 'spotify:album:31XXN8TRj92V1HB7Wq6LcL',
 'spotify:album:1ofMLjpy6bhkxpOcD5y436',
 'spotify:album:3YIO1RBJiQoGLimgK6XisN',
 'spotify:album:23MOdUJkZlG971DVuEINMr',
 'spotify:album:3Tm6OMfXk7PffAEt4NAkmX',
 'spotify:album:0T9tIRy1SjZTdTcMFhufu9',
 'spotify:album:63mOF4rcNutr1Aaj9QPgJK',
 'spotify:album:4fJeL4hdOewP135z0z0xhO',
 'spotify:album:0pBqLz20Olwl0JVODWwyoI',
 'spotify:album:5xPSi0aOjs3cbObM9ZTOd1',
 'spotify:album:6HlGzD3T7aL3Zr7w7QcNTn',
 'spotify:album:52FakbyRtaf6X8UzKxJfoh',
 'spotify:album:5EKDBIKZ37Idq0QiXG2qzk',
 'spotify:album:6cPoZUF1MLvQvXkfDGHwsf',
 'spotify:album:6ArjrXMlLegKiAPOp34K58',
 'spotify:album:4TAEAcuqfrU87gVTO6LoCP',
 'spotify:album:6mtyAknyjUbmwCP0Pjlk3k',
 'spotify:album:3MaBkywrJ1juBymiak7R2k',
 'spotify:album:4mBHxNHbABZPnckTpL2OSN',
 'spotify:album:1hxLe975qGYhzosF74wLxC',
 'spotify:album:4MuH5XuLZFuDc6kB2CSapu',
 'spotify:album:3lqqdqQPSFFujnoYAQbcKy',
 'spotify:album:166nVB36rcSGaol8O8AXfW',
 'spotify:album:

In [108]:
def get_track_id(album_ids):
    """Takes a list of spotify album id's, makes API call for track id's of that album"""
    result1 = []
    final_result = []
    for ids in album_ids:
        result1.append(spotify.album_tracks(f'{ids}'))
    for item in result1:
        for x in item['items']:
            final_result.append(x['uri'])
    return final_result

In [239]:
track_ids = get_track_id(album_id_1)
track_id_2 = get_track_id(album_id_2)

In [219]:
len(track_id_2)

453

In [198]:
len(track_id_1)

311

In [240]:
for ele in track_id_2:
    track_ids.append(ele)

In [241]:
len(track_ids)

764

In [149]:
for i in track_id_test:
    if i == 'spotify:track:s':
        print(i)
    

In [148]:
print(track_id_test)

['spotify:track:0Jt58oonOn1kfMh4VDRZ79', 'spotify:track:7M4SEotnv7t6ZqIKLH5dh0', 'spotify:track:2Smc8bQ5uHgiA6xsYRojgp', 'spotify:track:2rjLe6v56r7NvdJTpGDXwy', 'spotify:track:6gbar40Tl6IvpykjSmieJn', 'spotify:track:7K5T1XTl15X3JQ66NcRvZL', 'spotify:track:2xHLisbaTYlcgXABfLyzFy', 'spotify:track:4ZczlLzGRoTdNotmSDc4nr', 'spotify:track:6ZYqsJnY6wWoLn3kECuTSv', 'spotify:track:5UUKThuiVWNxKVsbOgkyWP', 'spotify:track:2K2fKcuM3xdOF1VZIXWmxA', 'spotify:track:1oTSNSx3Afa5gO9G7RX2hL', 'spotify:track:02OOv8jhfcww6sft29LRuS', 'spotify:track:7AwiTLNZuYKAtVvawiXou5', 'spotify:track:6z4J50cydm4UL2wLqk5W2L', 'spotify:track:1sqhbavNWew2aYvJ0wx2Lu', 'spotify:track:0L2C5LM439nlOhgKLThecR', 'spotify:track:2fQq5tkecQxNEBPYE9350M', 'spotify:track:1a0hlOegySMOBfxnUeHVgn', 'spotify:track:0FWcY55VzQYitIaLLa8I2M', 'spotify:track:45Hu3g2FFJBldkPyrhGuPl', 'spotify:track:3cg3AWFNOPpwySd3kljsE1', 'spotify:track:1kgS3qRiCPNLMndQQJA10A', 'spotify:track:2pl8P9WuTyXyMNde9cb6yN', 'spotify:track:4lCNjhpjPACGSLcT6Rlxlv',

In [150]:
def tracks_to_playlist(list_of_tracks):
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        results = sp.user_playlist_add_tracks(user = '1257267708', playlist_id = '75yDQp4uFy8iUyyMCQ8IZ3' , tracks = list_of_tracks)
        print(results)
    else:
            print("Can't get token for", username)
    return results    

In [181]:
tracks_to_playlist(track_id_test)

{'snapshot_id': 'MjcsMjVkOTQ1MjVhMDEyOGE2NmViMzE1MzVlMGY5ZjViMzU4OTJkM2Q4NA=='}


{'snapshot_id': 'MjcsMjVkOTQ1MjVhMDEyOGE2NmViMzE1MzVlMGY5ZjViMzU4OTJkM2Q4NA=='}

In [242]:
def remove_tracks(track_id_list):
    sp = spotipy.Spotify(auth=token)
    sp.user_playlist_remove_all_occurrences_of_tracks(user = '1257267708', playlist_id = '75yDQp4uFy8iUyyMCQ8IZ3' , tracks = track_id_list)




In [243]:
for i in range(0, len(track_ids), 100):
            remove_tracks(track_ids[i:i+100])

In [ ]:
#while there are still tracks in the track list

#add in increments of 100
#until count is equal to the track list

In [231]:

for i in range(0, len(track_ids), 100):
    print(track_ids[i:i+100])

['spotify:track:0Jt58oonOn1kfMh4VDRZ79', 'spotify:track:7M4SEotnv7t6ZqIKLH5dh0', 'spotify:track:2Smc8bQ5uHgiA6xsYRojgp', 'spotify:track:2rjLe6v56r7NvdJTpGDXwy', 'spotify:track:6gbar40Tl6IvpykjSmieJn', 'spotify:track:7K5T1XTl15X3JQ66NcRvZL', 'spotify:track:2xHLisbaTYlcgXABfLyzFy', 'spotify:track:4ZczlLzGRoTdNotmSDc4nr', 'spotify:track:6ZYqsJnY6wWoLn3kECuTSv', 'spotify:track:5UUKThuiVWNxKVsbOgkyWP', 'spotify:track:2K2fKcuM3xdOF1VZIXWmxA', 'spotify:track:1oTSNSx3Afa5gO9G7RX2hL', 'spotify:track:02OOv8jhfcww6sft29LRuS', 'spotify:track:7AwiTLNZuYKAtVvawiXou5', 'spotify:track:6z4J50cydm4UL2wLqk5W2L', 'spotify:track:1sqhbavNWew2aYvJ0wx2Lu', 'spotify:track:0L2C5LM439nlOhgKLThecR', 'spotify:track:2fQq5tkecQxNEBPYE9350M', 'spotify:track:1a0hlOegySMOBfxnUeHVgn', 'spotify:track:0FWcY55VzQYitIaLLa8I2M', 'spotify:track:45Hu3g2FFJBldkPyrhGuPl', 'spotify:track:3cg3AWFNOPpwySd3kljsE1', 'spotify:track:1kgS3qRiCPNLMndQQJA10A', 'spotify:track:2pl8P9WuTyXyMNde9cb6yN', 'spotify:track:4lCNjhpjPACGSLcT6Rlxlv',

In [234]:
track_ids[0]

'spotify:track:0Jt58oonOn1kfMh4VDRZ79'

In [244]:
for i in range(0, len(track_ids), 100):
            tracks_to_playlist(track_ids[i:i+100])

{'snapshot_id': 'NTMsNzMyOTcwZjM0MzMzMTc4MTU0NzhlNWM3ZWY3YTQ0YmI2MWEwOWJmNw=='}
{'snapshot_id': 'NTQsOGJjZjE4MjUzNTdkNGE3ZjJiOTQ0NGUyZTkzOGVkMzlkNDI4YjMyNQ=='}
{'snapshot_id': 'NTUsZTAyZmNmNDhhNDBmYzFiZjUwMDYwZTE1ZWVlNDZkZWI4M2JiODE4YQ=='}
{'snapshot_id': 'NTYsYTI2ZDE1NmE0OGVlNzdiZjdiMmQwZGVmYWEwZjlkNTJiMDA4YWQwYg=='}
{'snapshot_id': 'NTcsNDEyYmFlMTk0M2FhNWNjMzcwY2JjYjdjZmMwYjYyYWExNzg4ZTM0OA=='}
{'snapshot_id': 'NTgsMzFhYmE2MDQ1ZDMwZTYxNzhjZDE1YTFhNjgyZDEyOTUwNzkwYmM1Yg=='}
{'snapshot_id': 'NTksMDg0ZDJhYzBhZmNkYmM4NGJhZWEzYzQ5NjczN2NjYzMyZGE3NWU3NQ=='}
{'snapshot_id': 'NjAsNzg3YjljNjIzMTY0M2MxMGJhN2IxMDQxZDMxYWZjZTkzMDhkYzRiZQ=='}


In [232]:
count = 0 

while count < len(track_ids):
    try:
        for i in range(0, len(track_ids), 100):
            tracks_to_playlist(track_ids[i:i+100])
            count += 100
    if count == len(track_ids):
        break

SyntaxError: invalid syntax (<ipython-input-232-0588db6c6465>, line 8)